In [1]:

# Import libraries
import betfairlightweight

from betfairlightweight import filters
from datetime import datetime
from datetime import timedelta
from dateutil import tz

import math
import numpy as np
import pandas as pd

from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft


In [28]:


race_details = pd.read_csv("C:\\Users\\karan\\Documents\\Data\\racing\\race_details_20210101_20220424.csv")

dog_results = pd.read_csv("C:\\Users\\karan\\Documents\\Data\\racing\\dog_results_20210101_20220424.csv")

dog_results.rename(columns = {"@id":"dogid"}, inplace = True)
dog_results['dogid'] = dog_results['dogid'].astype('category')
dog_results['f_win'] = dog_results.apply(lambda x : 1 if x.Place == '1' else 0 , axis =1 )


In [33]:

print(race_details.shape)
print(race_details.columns.values.tolist())

print()

print(dog_results.shape)
print(dog_results.columns.values.tolist())


(8949, 8)
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']

(79464, 20)
['dogid', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', 'f_win']


In [74]:

#print(dog_results.dtypes)


In [35]:

dog_results.head()


,dogid,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,f_win
0,460023770,1,SAKKARI,4.0,4,28.0,$3.20F,NaN,1.75,NaN,NaN,0,NaN,NaN,17.14,1400.0,626271122,66176,A Monaghan,1
1,407727859,2,BANJO LAD,7.0,7,30.6,$12.00,NaN,1.75,1.71,NaN,0,NaN,NaN,17.26,400.0,626271122,275881,N Evans,0
2,450198101,3,OHANA LEA,5.0,5,25.1,$5.00,NaN,2.50,1.00,NaN,0,NaN,NaN,17.33,200.0,626271122,83303,M Heilbronn,0
3,323655139,4,CLONMORE,2.0,2,25.4,$8.50,NaN,3.75,1.14,NaN,0,NaN,NaN,17.41,100.0,626271122,61234,B Kitchener,0
4,393648268,5,AURIGA RACHEL,8.0,9,28.9,$26.00,NaN,5.25,1.43,NaN,0,NaN,NaN,17.51,0.0,626271122,105852,G Draheim,0


In [5]:

# Fitting a Model


In [36]:

#from statsmodels import ConditionalLogit 

from statsmodels.discrete.conditional_models import ConditionalLogit 


In [73]:

#dog_results.info()


In [41]:

print(dog_results.shape)

df_model = dog_results[['dogid','f_win','Weight','Box','Rug']].dropna()

print(df_model.shape)


(79464, 20)
(66480, 5)


In [60]:

from sklearn.model_selection import train_test_split

y = df_model['f_win']
X = df_model[ ['dogid','Weight','Box','Rug'] ]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


gp_train = x_train['dogid']
gp_test = x_test['dogid']

x_train.drop(columns = ['dogid'], inplace = True)
x_test.drop(columns = ['dogid'], inplace = True)

print(f'x_train:{x_train.shape}, x_test:{x_test.shape}, y_train:{y_train.shape}, y_test:{y_test.shape}')

print(f'gp_train:{gp_train.shape}, gp_test:{gp_test.shape}')



x_train:(49860, 3), x_test:(16620, 3), y_train:(49860,), y_test:(16620,)
gp_train:(49860,), gp_test:(16620,)


In [61]:

print(type(y))
print(type(X))
print(type(groups))

model = ConditionalLogit(y_train, x_train, groups = gp_train)


<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\conditional_models.py:80: UserWarning: Dropped 1805 groups and 9512 observations for having no within-group variance
  warnings.warn(msg)


In [63]:

print(type(model))

print(model.endog_names)
print(model.exog_names)


<class 'statsmodels.discrete.conditional_models.ConditionalLogit'>
f_win
['Weight', 'Box', 'Rug']


In [64]:

model.fit()


In [78]:

print(model.summary)


AttributeError: 'ConditionalLogit' object has no attribute 'summary'

In [65]:

print(model.hessian)


<bound method _ConditionalModel.hessian of <statsmodels.discrete.conditional_models.ConditionalLogit object at 0x000002C54765CC10>>


In [72]:

type(model.predict)
#metho aka function


method

In [77]:


model.score(x_test)


ValueError: shapes (27,3) and (16620,3) not aligned: 3 (dim 1) != 16620 (dim 0)